<a href="https://colab.research.google.com/github/ManasviAtGitHub/Natural-Language-Processing/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
cd drive/MyDrive/NLP/datasets/

/content/drive/MyDrive/NLP/datasets


In [5]:
!pip install tflearn

     |████████████████████████████████| 107 kB 5.4 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=f1d79a56884111adef54a206578a243d4b306c34e6a793f28c0f9cfbcc06e437
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [6]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy
import tflearn
import tensorflow
import random
import json
import pickle

Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [29]:
stemmer = LancasterStemmer()

In [30]:
words = []
labels = []
docs_x = []
docs_y = []

In [31]:
with open("intents.json") as file:
    data = json.load(file)

In [32]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
    print(f'docs_x: {docs_x}, docs_y: {docs_y}, Words: {words}')


    if intent["tag"] not in labels:
        labels.append(intent["tag"])
    print(f'labels: {labels}')
    
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))
labels = sorted(labels)
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

docs_x: [['Hi'], ['How', 'are', 'you'], ['Is', 'anyone', 'there', '?'], ['Hello'], ['Good', 'day']], docs_y: ['greeting', 'greeting', 'greeting', 'greeting', 'greeting'], Words: ['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day']
labels: ['greeting']
docs_x: [['Hi'], ['How', 'are', 'you'], ['Is', 'anyone', 'there', '?'], ['Hello'], ['Good', 'day'], ['Bye'], ['See', 'you', 'later'], ['Goodbye']], docs_y: ['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye'], Words: ['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye']
labels: ['greeting', 'goodbye']
docs_x: [['Hi'], ['How', 'are', 'you'], ['Is', 'anyone', 'there', '?'], ['Hello'], ['Good', 'day'], ['Bye'], ['See', 'you', 'later'], ['Goodbye'], ['Thanks'], ['Thank', 'you'], ['That', "'s", 'helpful']], docs_y: ['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye',

In [33]:
words

["'s",
 'acceiv',
 'anyon',
 'ar',
 'bye',
 'card',
 'cash',
 'credit',
 'day',
 'do',
 'good',
 'goodby',
 'hello',
 'help',
 'hi',
 'hour',
 'how',
 'is',
 'lat',
 'mastercard',
 'on',
 'op',
 'see',
 'tak',
 'thank',
 'that',
 'ther',
 'today',
 'what',
 'when',
 'yo',
 'you']

In [34]:
out_empty

[0, 0, 0, 0, 0, 0]

In [35]:
for x, doc in enumerate(docs_x):
  print(f'x : {x}, doc : {doc}')

x : 0, doc : ['Hi']
x : 1, doc : ['How', 'are', 'you']
x : 2, doc : ['Is', 'anyone', 'there', '?']
x : 3, doc : ['Hello']
x : 4, doc : ['Good', 'day']
x : 5, doc : ['Bye']
x : 6, doc : ['See', 'you', 'later']
x : 7, doc : ['Goodbye']
x : 8, doc : ['Thanks']
x : 9, doc : ['Thank', 'you']
x : 10, doc : ['That', "'s", 'helpful']
x : 11, doc : ['What', 'hours', 'are', 'you', 'open', '?']
x : 12, doc : ['What', 'are', 'your', 'hours', '?']
x : 13, doc : ['When', 'are', 'you', 'open', '?']
x : 14, doc : ['Do', 'you', 'take', 'credit', 'cards', '?']
x : 15, doc : ['Do', 'you', 'accept', 'Mastercard', '?']
x : 16, doc : ['Are', 'you', 'cash', 'only', '?']
x : 17, doc : ['Are', 'you', 'open', 'today', '?']
x : 18, doc : ['When', 'do', 'you', 'open', 'today', '?']
x : 19, doc : ['What', 'are', 'your', 'hours', 'today', '?']


In [36]:
orow = out_empty[:]

In [37]:
print(orow)

[0, 0, 0, 0, 0, 0]


In [38]:
for x, doc in enumerate(docs_x):
  bag = []
  wrds = [stemmer.stem(w.lower()) for w in doc]
  for w in words:
      if w in wrds:
          bag.append(1)
      else:
          bag.append(0)

  output_row = out_empty[:]
  output_row[labels.index(docs_y[x])] = 1
  ##docs_y[x]
  ## x = 0
  ## docs_y[0] ==>> 'greeting'
  ##labels.index('greetings')==>0
  ## output_row [0] = 1

  training.append(bag)
  output.append(output_row)


In [40]:
print(f'Training : {training[0]}, Output: {output[0]}')

Training : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Output: [0, 1, 0, 0, 0, 0]


In [41]:
training = numpy.array(training)
output = numpy.array(output)

In [45]:

tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

In [46]:
model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
# model.save("model.tflearn")

Training Step: 2999  | total loss: 0.01452 | time: 0.022s
| Adam | epoch: 1000 | loss: 0.01452 - acc: 1.0000 -- iter: 16/20
Training Step: 3000  | total loss: 0.01429 | time: 0.029s
| Adam | epoch: 1000 | loss: 0.01429 - acc: 1.0000 -- iter: 20/20
--


In [47]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

In [48]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        print(results)
        results_index = numpy.argmax(results)
        print(results_index)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

In [49]:
chat()

Start talking with the bot (type quit to stop)!
You: hi, how are you ?
[[1.7634567e-04 9.9905640e-01 3.2901691e-04 5.6016092e-06 1.3389500e-07
  4.3241153e-04]]
1
Hi there, how can I help?
You: Is your shop open
[[0.02846025 0.44413322 0.44312474 0.06523647 0.00263152 0.01641374]]
1
Hello, thanks for visiting
You: are you open 
[[0.02354181 0.06451001 0.8383112  0.06879287 0.00222847 0.0026156 ]]
2
Our hours are 9am-9pm every day
You: hello good morning
[[6.4024492e-04 9.9840564e-01 1.0208351e-04 1.6051857e-06 3.3580361e-07
  8.5012463e-04]]
1
Good to see you again
You: quit
